In [ ]:
POWER_CAPACITY = 10 # MW

LOAD_FACTOR = 0.9 


POWER_PER_YEAR = POWER_CAPACITY * LOAD_FACTOR * 365 * 24  

In [ ]:
f"Power per year is {POWER_PER_YEAR} MWh" # Joules

'Power per year is 283824000.0 MWh'